In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uom190346a/sleep-health-and-lifestyle-dataset")

print("Path to dataset files:", path)

#https://www.kaggle.com/code/jillanisofttech/sleep-health-and-lifestyle-predication-with-94-ac
#eksempel av noen sin løsning på dette datasetet

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(path+"/Sleep_health_and_lifestyle_dataset.csv")

In [ ]:
headAmount = 5 # for å skrive ut alle data punkt i csv file skriv data.size() istedenfor eit tall
data.head(headAmount)

In [ ]:
print(data['Sleep Disorder'].unique())
print(data['Occupation'].unique())
print(data['BMI Category'].unique())

In [ ]:
data['Blood Pressure'].unique()

In [ ]:
# Vi deler opp blodtrykk siden det er to verdiar i ein kolonne
data_16_pro_xl = (pd.concat([data, data['Blood Pressure'].str.split('/', expand=True)], axis=1).
                  drop('Blood Pressure', axis=1))

In [ ]:
#data_16_pro_xl

In [ ]:
data_16_pro_xl = data_16_pro_xl.rename(columns={0: 'Systolic Blood Pressure', 1: 'Diastolic Blood Pressure'})

In [ ]:
#data_16_pro_xl

In [ ]:
data_16_pro_xl['Systolic Blood Pressure'] = data_16_pro_xl['Systolic Blood Pressure'].astype(float)
data_16_pro_xl['Diastolic Blood Pressure'] = data_16_pro_xl['Diastolic Blood Pressure'].astype(float)
data_16_pro_xl.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import json
from pathlib import Path

# Fit encoders on full data so all categories are captured
categoricals = ["Gender", "Occupation", "BMI Category"]
enc_map = {}
for col in categoricals:
    le = LabelEncoder()
    data_16_pro_xl[col] = le.fit_transform(data_16_pro_xl[col])
    enc_map[col] = {str(cls): int(code) for cls, code in zip(le.classes_, le.transform(le.classes_))}

# Target label order for decoding predictions later
le_y = LabelEncoder()
data_16_pro_xl['Sleep Disorder'] = le_y.fit_transform(data_16_pro_xl['Sleep Disorder'])
target_classes = [str(c) for c in le_y.classes_]

Path("encoders.json").write_text(json.dumps(enc_map, indent=2))
Path("target_classes.json").write_text(json.dumps(target_classes, indent=2))


In [ ]:
X = data_16_pro_xl.drop(['Sleep Disorder','Person ID'], axis=1)
y = data_16_pro_xl['Sleep Disorder']

In [ ]:
import joblib
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}")

joblib.dump(model, 'sleep_disorder_model.pkl')